In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/Amex/

Models	Predictions	 test.parquet	   train_labels.csv
OOF	test_fe.parquet  train_fe.parquet  train.parquet


In [4]:
!pip3 install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 48.6 MB/s 
     |████████████████████████████████| 81 kB 9.6 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 112 kB 65.2 MB/s 
     |████████████████████████████████| 147 kB 61.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=6b69b702463d60196375451ab2a783f6ac59daf6a9a109015fa14be403bd0436
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


# Preprocessing

In [5]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '/content/drive/MyDrive/Amex/'
    seed = 42
    #seed = 52
    #seed = 62
    n_folds = 5
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'
    #metric = None

# ====================================================
# Seed everything
# ====================================================
# 実験用に乱数seedを固定する
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
# 保存した素性付きデータを読み込む
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def xgb_amex_metric(y_pred: np.ndarray, dtrain: xgb.DMatrix):
    y_true = dtrain.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred)

In [6]:
seed_everything(CFG.seed)
train, test = read_data()

In [7]:
# importanceが0のものを弾いておく
IGNORE_COL = ['B_31_max',
 'D_108_min',
 'D_109_diff1',
 'D_109_last',
 'D_109_max',
 'D_111_diff1',
 'D_111_min',
 'D_123_min',
 'D_127_min',
 'D_137_min',
 'D_138_min',
 'D_65_min',
 'D_86_diff1',
 'D_87_diff1',
 'D_87_last',
 'D_87_max',
 'D_87_mean',
 'D_87_min',
 'D_88_diff1',
 'D_92_min',
 'D_93_diff1',
 'D_93_last',
 'D_93_max',
 'D_93_min',
 'D_94_diff1',
 'D_94_last',
 'D_94_min',
 'D_96_min',
 'R_10_min',
 'R_13_min',
 'R_14_min',
 'R_15_min',
 'R_17_min',
 'R_18_diff1',
 'R_18_last',
 'R_18_last_mean_diff',
 'R_18_max',
 'R_18_mean',
 'R_18_min',
 'R_18_std',
 'R_19_diff1',
 'R_20_min',
 'R_21_min',
 'R_22_min',
 'R_23_diff1',
 'R_23_last',
 'R_23_last_mean_diff',
 'R_23_mean',
 'R_23_min',
 'R_24_min',
 'R_25_last',
 'R_25_min',
 'R_26_diff1',
 'R_28_diff1',
 'R_28_last',
 'R_28_last_mean_diff',
 'R_28_min',
 'R_4_min',
 'R_5_min',
 'R_7_min',
 'R_8_min',
 'S_18_last',
 'S_18_min',
 'S_20_min',]

In [8]:
#test = test.head(10000)
#train = train.head(10000)

In [9]:
"""
## 読み込んだcvsからスコアを計算できるかチェック
#xgb = pd.read_csv('/content/drive/MyDrive/Amex/OOF/oof_xgb_dart_baseline_5fold_seed42_tmp.csv')
cat = pd.read_csv('/content/drive/MyDrive/Amex/OOF/oof_catboost_dart_baseline_5fold_seed42_tmp.csv')
lgbm = pd.read_csv('/content/drive/MyDrive/Amex/OOF/oof_lgbm_dart_baseline_5fold_seed42_tmp.csv')
W = 0.5
ensemble = (W * cat['prediction'] + (1-W) * lgbm['prediction'])
# xgb-only 0.7933054247444402
# lgbm only 0.7976190229811759
# 0.1 0.7975738205453153
# 0.2 0.7978333342120119 一番?
# 0.3 0.7978847561862471
# 0.4 0.7977746617727102
# 0.5 0.797406325414312
# 0.9 0.7943135729314539

# LGBM + catboost
# cat 0.1 : 0.7978951298126327
# cat 0.2 : 0.797994130906287
# cat 0.3 : 0.7982869115898213 BEST
# cat 0.5 : 0.7980073124190881
# cat 0.7 : 0.7974678609833052
# cat 1.0 : 0.7956859228104673

score = amex_metric(train[CFG.target], ensemble)
score
"""

0.7956859228104673

# Optuna
- 使わない場合は全部コメントアウト

In [ ]:
import optuna

In [ ]:
kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
x_train, x_val, y_train, y_val = None, None, None, None
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
    print(' ')
    print('-'*50)
    if fold == 1:
        break
    # xはfeature, yはtarget
    print(f'Training fold {fold} with {len(features)} features...')
    x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
    y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]

In [ ]:
def objective(trial):
    eta =  trial.suggest_loguniform('eta', 0.001, 0.5)
    gamma = trial.suggest_loguniform('gamma', 1e-6, 1e-4)
    max_depth = trial.suggest_int('max_depth', 5, 20)
    min_child_weight = trial.suggest_loguniform('min_child_weight', 1e-3, 0.1)
    max_delta_step = trial.suggest_loguniform('max_delta_step', 0.1, 1.0)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
    reg_lambda = trial.suggest_uniform('reg_lambda', 200.0, 1000.0)
    reg_alpha = trial.suggest_uniform('reg_alpha', 0.0, 100.0)


    model =xgb.XGBClassifier(
        tree_method      = 'gpu_hist', # GPUを使う
        booster          = 'gbtree',
        objective='binary:logistic',
        eval_metric='logloss', 
        #eval_metric=xgb_amex_metric,
        eta = eta, 
        gamma = gamma, 
        max_depth = max_depth,
        min_child_weight = min_child_weight, 
        max_delta_step = max_delta_step,
        subsample = subsample,
        reg_lambda = reg_lambda,
        n_estimators = 6000,
        reg_alpha = reg_alpha
        )
    #scoring = sklearn.metrics.make_scorer(sklearn.metrics.map_score, greater_is_better=True)
    model.fit(x_train, y_train, 
        eval_set=[(x_val,y_val)],
        verbose=50,
        #eval_metric=xgb_amex_metric,
        eval_metric='logloss',
        early_stopping_rounds=150
    )

    return model.get_booster().best_score

In [ ]:
# optunaで最適値を見つける
#study = optuna.create_study(direction='maximize')
#study = optuna.create_study(direction='minimize')
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2022-07-20 17:09:28,137] A new study created in memory with name: no-name-0696527c-1841-4bc1-9a9d-593dff90b2a8


[0]	validation_0-logloss:0.685305
Will train until validation_0-logloss hasn't improved in 150 rounds.
[50]	validation_0-logloss:0.406543
[100]	validation_0-logloss:0.285563
[150]	validation_0-logloss:0.241764
[200]	validation_0-logloss:0.226783
[250]	validation_0-logloss:0.221422
[300]	validation_0-logloss:0.219111
[350]	validation_0-logloss:0.218021
[400]	validation_0-logloss:0.217309
[450]	validation_0-logloss:0.21677
[500]	validation_0-logloss:0.216389
[550]	validation_0-logloss:0.216094
[600]	validation_0-logloss:0.215874
[650]	validation_0-logloss:0.215702
[700]	validation_0-logloss:0.215497
[750]	validation_0-logloss:0.215446
[800]	validation_0-logloss:0.215412
[850]	validation_0-logloss:0.215355
[900]	validation_0-logloss:0.215321
[950]	validation_0-logloss:0.215258
[1000]	validation_0-logloss:0.215272
[1050]	validation_0-logloss:0.215268
[1100]	validation_0-logloss:0.215267
[1150]	validation_0-logloss:0.21527
Stopping. Best iteration:
[1033]	validation_0-logloss:0.215211



[I 2022-07-20 17:11:58,264] Trial 0 finished with value: 0.215211 and parameters: {'eta': 0.024617659033885325, 'gamma': 8.017969222527678e-06, 'max_depth': 18, 'min_child_weight': 0.008423628186462232, 'max_delta_step': 0.21177074285380784, 'subsample': 0.9259349574136867, 'reg_lambda': 944.3556288361682, 'reg_alpha': 40.170498523267504}. Best is trial 0 with value: 0.215211.


[0]	validation_0-logloss:0.682217
Will train until validation_0-logloss hasn't improved in 150 rounds.
[50]	validation_0-logloss:0.344339
[100]	validation_0-logloss:0.249373
[150]	validation_0-logloss:0.228049
[200]	validation_0-logloss:0.222676
[250]	validation_0-logloss:0.220475
[300]	validation_0-logloss:0.219145
[350]	validation_0-logloss:0.218286
[400]	validation_0-logloss:0.217717
[450]	validation_0-logloss:0.217251
[500]	validation_0-logloss:0.216874
[550]	validation_0-logloss:0.216588
[600]	validation_0-logloss:0.216425
[650]	validation_0-logloss:0.216242
[700]	validation_0-logloss:0.216101
[750]	validation_0-logloss:0.215962
[800]	validation_0-logloss:0.21591
[850]	validation_0-logloss:0.215812
[900]	validation_0-logloss:0.215685
[950]	validation_0-logloss:0.215648
[1000]	validation_0-logloss:0.215625
[1050]	validation_0-logloss:0.215641
[1100]	validation_0-logloss:0.215562
[1150]	validation_0-logloss:0.215523
[1200]	validation_0-logloss:0.215491
[1250]	validation_0-logloss:0.

[I 2022-07-20 17:14:12,028] Trial 1 finished with value: 0.215471 and parameters: {'eta': 0.4245053226231726, 'gamma': 3.560994232093194e-06, 'max_depth': 17, 'min_child_weight': 0.0022441956898192586, 'max_delta_step': 0.2968828753279279, 'subsample': 0.9650008135347001, 'reg_lambda': 653.7090508595746, 'reg_alpha': 90.21243448038825}. Best is trial 0 with value: 0.215211.


...

[I 2022-07-20 20:33:38,075] Trial 97 finished with value: 0.214478 and parameters: {'eta': 0.004418943455036788, 'gamma': 4.588548573580885e-06, 'max_depth': 6, 'min_child_weight': 0.0016009284957516346, 'max_delta_step': 0.6881240927859895, 'subsample': 0.9521025458225673, 'reg_lambda': 953.8677305959843, 'reg_alpha': 18.501267370254137}. Best is trial 87 with value: 0.21392.


[0]	validation_0-logloss:0.662801
Will train until validation_0-logloss hasn't improved in 150 rounds.
[50]	validation_0-logloss:0.243173
[100]	validation_0-logloss:0.227794
[150]	validation_0-logloss:0.222936
[200]	validation_0-logloss:0.220521
[250]	validation_0-logloss:0.219132
[300]	validation_0-logloss:0.218089
[350]	validation_0-logloss:0.217419
[400]	validation_0-logloss:0.21692
[450]	validation_0-logloss:0.216462
[500]	validation_0-logloss:0.216123
[550]	validation_0-logloss:0.215861
[600]	validation_0-logloss:0.215643
[650]	validation_0-logloss:0.215493
[700]	validation_0-logloss:0.215296
[750]	validation_0-logloss:0.215156
[800]	validation_0-logloss:0.215054
[850]	validation_0-logloss:0.214933
[900]	validation_0-logloss:0.214845
[950]	validation_0-logloss:0.214781
[1000]	validation_0-logloss:0.214695
[1050]	validation_0-logloss:0.214601
[1100]	validation_0-logloss:0.214546
[1150]	validation_0-logloss:0.214504
[1200]	validation_0-logloss:0.214448
[1250]	validation_0-logloss:0.

[I 2022-07-20 20:35:36,722] Trial 98 finished with value: 0.214243 and parameters: {'eta': 0.003949284309865579, 'gamma': 1.152082874166337e-05, 'max_depth': 5, 'min_child_weight': 0.00201092602844923, 'max_delta_step': 0.8554057533590634, 'subsample': 0.9367297000811327, 'reg_lambda': 983.453502207809, 'reg_alpha': 25.70147737239279}. Best is trial 87 with value: 0.21392.


[0]	validation_0-logloss:0.66244
Will train until validation_0-logloss hasn't improved in 150 rounds.
[50]	validation_0-logloss:0.24285
[100]	validation_0-logloss:0.227529
[150]	validation_0-logloss:0.222749
[200]	validation_0-logloss:0.220279
[250]	validation_0-logloss:0.218865
[300]	validation_0-logloss:0.217937
[350]	validation_0-logloss:0.21729
[400]	validation_0-logloss:0.216812
[450]	validation_0-logloss:0.216428
[500]	validation_0-logloss:0.216109
[550]	validation_0-logloss:0.215883
[600]	validation_0-logloss:0.215604
[650]	validation_0-logloss:0.215431
[700]	validation_0-logloss:0.215259
[750]	validation_0-logloss:0.215076
[800]	validation_0-logloss:0.214974
[850]	validation_0-logloss:0.214874
[900]	validation_0-logloss:0.214766
[950]	validation_0-logloss:0.214717
[1000]	validation_0-logloss:0.214684
[1050]	validation_0-logloss:0.214641
[1100]	validation_0-logloss:0.214659
[1150]	validation_0-logloss:0.214603
[1200]	validation_0-logloss:0.214532
[1250]	validation_0-logloss:0.21

[I 2022-07-20 20:37:39,344] Trial 99 finished with value: 0.214377 and parameters: {'eta': 0.003781297991662751, 'gamma': 1.1217129346523395e-05, 'max_depth': 5, 'min_child_weight': 0.0015713071573753709, 'max_delta_step': 0.8663002829739659, 'subsample': 0.9312955566536258, 'reg_lambda': 970.8908591950664, 'reg_alpha': 25.39524057283621}. Best is trial 87 with value: 0.21392.


In [ ]:
 """

    # デバッグ
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        if fold == 1:
            break
        # xはfeature, yはtarget
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]


        # 学習、評価
        model = xgb.XGBClassifier( 
        n_estimators=10000,
        tree_method      = 'gpu_hist', # GPUを使う
        booster          = 'gbtree',
        objective='binary:logistic',
        eval_metric='logloss',
        #eval_metric=xgb_amex_metric,
        eta = 0.004986959396603572,
        gamma = 4.370438625953378e-06, 
        max_depth = 5, 
        min_child_weight = 0.003205746269737129, 
        max_delta_step = 0.8640647607358845, 
        subsample = 0.9791072895202135, 
        reg_lambda = 932.5355776903212, 
        reg_alpha = 22.366140749569205,
        #colsample_bytree=0.4, 
        #missing=np.nan, 
        #eval_metric=['logloss', xgb_amex_metric],
        )
        
        model.fit(x_train, y_train, 
        eval_set=[(x_val,y_val)],
        verbose=50,
        eval_metric='logloss',
        #eval_metric=xgb_amex_metric,
        #eval_metric='logloss',
        early_stopping_rounds=150
        )


        #del xgb_train, xgb_valid
        #gc.collect()
        # Save best model
        # このコードだとmodelは書き出しのみで読み込んではいない？
        #joblib.dump(model, f'/content/drive/MyDrive/Amex/Models/xgb_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
        """

 
--------------------------------------------------
Training fold 0 with 1765 features...
[0]	validation_0-logloss:0.662471
Will train until validation_0-logloss hasn't improved in 150 rounds.
[50]	validation_0-logloss:0.242582
[100]	validation_0-logloss:0.227457
[150]	validation_0-logloss:0.222742
[200]	validation_0-logloss:0.220385
[250]	validation_0-logloss:0.218962
[300]	validation_0-logloss:0.21805
[350]	validation_0-logloss:0.217369
[400]	validation_0-logloss:0.216837
[450]	validation_0-logloss:0.216454
[500]	validation_0-logloss:0.216143
[550]	validation_0-logloss:0.215865
[600]	validation_0-logloss:0.2157
[650]	validation_0-logloss:0.215541
[700]	validation_0-logloss:0.215337
[750]	validation_0-logloss:0.215157
[800]	validation_0-logloss:0.215049
[850]	validation_0-logloss:0.214907
[900]	validation_0-logloss:0.214811
[950]	validation_0-logloss:0.214686
[1000]	validation_0-logloss:0.214622
[1050]	validation_0-logloss:0.214564
[1100]	validation_0-logloss:0.214511
[1150]	validati

In [ ]:
# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    # lastはcategoricalそのものなので指定する
    cat_features = [f"{cf}_last" for cf in cat_features]
    # cat_featuresで指定するならlabel encodingしなくてもよさそう？
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    # lastのfeatureはround(2)でまとめたものを加える (そのほうがスコア上がるらしい)
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)


    # Get the difference between last and mean
    # 平均からのlastのずれを素性に加える
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
    # Transform float64 and float32 to float16
    # 容量節約
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)

    train = train.drop(IGNORE_COL , axis=1)
    test = test.drop(IGNORE_COL , axis=1)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]

    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    # ここからCV
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        # xはfeature, yはtarget
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
   
        # 学習、評価
        model = xgb.XGBClassifier( 
        n_estimators=10000,
        tree_method      = 'gpu_hist', # GPUを使う
        #booster          = 'gbtree',
        booster          = 'dart',
        objective='binary:logistic',
        eval_metric='logloss',
        #eval_metric=xgb_amex_metric,
        eta = 0.004986959396603572,
        gamma = 4.370438625953378e-06, 
        max_depth = 5, 
        min_child_weight = 0.003205746269737129, 
        max_delta_step = 0.8640647607358845, 
        subsample = 0.9791072895202135, 
        reg_lambda = 932.5355776903212, 
        reg_alpha = 22.366140749569205,
        #colsample_bytree=0.4, 
        #missing=np.nan, 
        #eval_metric=['logloss', xgb_amex_metric],
        )
        
        model.fit(x_train, y_train, 
        eval_set=[(x_val,y_val)],
        verbose=50,
        eval_metric='logloss',
        #eval_metric=xgb_amex_metric,
        #eval_metric='logloss',
        early_stopping_rounds=150
        )
        del x_train, y_train
        gc.collect()

        #del lgb_train, lgb_valid
        #gc.collect()
        # Save best model
        # このコードだとmodelは書き出しのみで読み込んではいない？
        joblib.dump(model, f'/content/drive/MyDrive/Amex/Models/xgb_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')

        # Predict validation
        # でかいので分割してpredict
        Nrow = x_val.shape[0]
        Ndiv = 5
        n = int(Nrow // Ndiv) + 1
        x_val_list = []
        for i in range(Ndiv):
            tmp = x_val.iloc[i*n : (i+1)*n, :]
            x_val_list.append(tmp)
        del x_val
        gc.collect()

        val_pred_list = [] 
        for i, v in enumerate(x_val_list):
            print('train pred i=', i)
            tmp = model.predict_proba(v, ntree_limit=model.get_booster().best_iteration)[:, 1]
            val_pred_list.append(tmp) 
        del x_val_list
        gc.collect()
        val_pred = np.concatenate(val_pred_list)
        del val_pred_list
        gc.collect()
        #val_pred = model.predict(x_val)

        # Add to out of folds array
        # CVを終えれば全部のindexが1回ずつ計算されることになる
        oof_predictions[val_ind] = val_pred


        # Predict the test set
        # でかいので分割してpredict
        Nrow = test[features].shape[0]
        n = int(Nrow // Ndiv) + 1
        x_test_list = []
        for i in range(Ndiv):
            tmp = test[features].iloc[i*n : (i+1)*n, :]
            x_test_list.append(tmp)
        #del test
        #gc.collect()

        test_pred_list = [] 
        for i, v in enumerate(x_test_list):
            print('test pred i=', i)
            tmp = model.predict_proba(v, ntree_limit=model.get_booster().best_iteration)[:, 1]
            test_pred_list.append(tmp) 
        del x_test_list
        gc.collect()
        test_pred = np.concatenate(test_pred_list)
        del test_pred_list
        gc.collect()
        #test_pred = model.predict(test[features])

        # 不要になった時点でモデル削除
        del model
        gc.collect()
        # testの場合はfoldの数で割る
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del y_val
        gc.collect()
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    # trainでcvして予測したやつ
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_xgb_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}_tmp.csv', index = False)
    # Create a dataframe to store test prediction
    # 提出用の大事なやつはこれ
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_xgb_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}_tmp.csv', index = False)
    
train_and_evaluate(train, test)

  0%|          | 0/1496 [00:00<?, ?it/s]

 
--------------------------------------------------
Training fold 0 with 1765 features...
[0]	validation_0-logloss:0.662471
Will train until validation_0-logloss hasn't improved in 150 rounds.
[50]	validation_0-logloss:0.242606
[100]	validation_0-logloss:0.227324
[150]	validation_0-logloss:0.222715
[200]	validation_0-logloss:0.220284
[250]	validation_0-logloss:0.218804
[300]	validation_0-logloss:0.217905
[350]	validation_0-logloss:0.217214
[400]	validation_0-logloss:0.216763
[450]	validation_0-logloss:0.216418
[500]	validation_0-logloss:0.216042
[550]	validation_0-logloss:0.215781
[600]	validation_0-logloss:0.215575
[650]	validation_0-logloss:0.21539
[700]	validation_0-logloss:0.215207
[750]	validation_0-logloss:0.215039
[800]	validation_0-logloss:0.214862
[850]	validation_0-logloss:0.214785
[900]	validation_0-logloss:0.214647
[950]	validation_0-logloss:0.214626
[1000]	validation_0-logloss:0.214539
[1050]	validation_0-logloss:0.214474
[1100]	validation_0-logloss:0.214403
[1150]	valida